In [4]:
import os, re, glob, json
from os.path import join as opj
import numpy as np
import pandas as pd

import networkx as nx

import ohol_transitions as transition
import ohol_categories as cat
import ohol_objects as obj

### Find object and transition files:

Transitions:

In [6]:
gsearch = lambda *args: glob.glob(opj(*args))
data_dir = '../../OneLifeData7/'
trans_dir  = opj(data_dir, 'transitions')
trans_files = gsearch(trans_dir, '*.txt')

print(*trans_files[:5], sep='\n')

../../OneLifeData7/transitions/-1_2574.txt
../../OneLifeData7/transitions/0_702.txt
../../OneLifeData7/transitions/314_235.txt
../../OneLifeData7/transitions/2165_2165.txt
../../OneLifeData7/transitions/0_1692.txt


Objects:

In [7]:
obj_dir = opj(data_dir, 'objects')
obj_files = gsearch(obj_dir, '*txt')

print(*obj_files[:5], sep='\n')

../../OneLifeData7/objects/3644.txt
../../OneLifeData7/objects/1053.txt
../../OneLifeData7/objects/1735.txt
../../OneLifeData7/objects/3122.txt
../../OneLifeData7/objects/2228.txt


### Helper Functions:

In [17]:
##helper function that finds the ingredients given the product from the transition dictionary
def find_ingredients(value):
    ingredient_list = []
    for idx in d.items():
        for item in d[idx[0]].items():
            if item[0] != 'category':
                if value in item[1]:
                    ingredient_list.append(item[0])
    return list(set(ingredient_list))       


In [20]:
##helper function that returns the category for a given object
def find_parent(mydict, child):
    for i in mydict.keys():
        if child in mydict[i]:
            return i

In [641]:
def split_dataframe_rows(df,column_selectors):
    # we need to keep track of the ordering of the columns
    def _split_list_to_rows(row,row_accumulator,column_selector):
        split_rows = {}
        max_split = 0
        for column_selector in column_selectors:
            split_row = row[column_selector]
            split_rows[column_selector] = split_row
            if len(split_row) > max_split:
                max_split = len(split_row)
        for i in range(max_split):
            new_row = row.to_dict()
            for column_selector in column_selectors:
                try:
                    new_row[column_selector] = split_rows[column_selector].pop(0)
                except IndexError:
                    new_row[column_selector] = ''
            row_accumulator.append(new_row)

    new_rows = []
    df.apply(_split_list_to_rows,axis=1,args = (new_rows,column_selectors))
    new_df = pd.DataFrame(new_rows, columns=df.columns)
    return new_df

In [19]:
##Helper function that finds the key associated with (/smaller than) a certain value in the dictionary
def find_value(dictionary, value, multiple):
    mylist = []
    for item in dictionary.items():
        if multiple == 0:
            if value == item[1]:
                mylist.append(item[0])
        elif multiple == 1:
            if value > item[1]:
                mylist.append(item[0])
    return mylist

Build a dictionary of {object: mapChance} pairs, and identify grave object

In [222]:
str_extract = lambda pattern, s: re.search(pattern, s).group(0)
int_extract = lambda pattern, s: int(str_extract(pattern, s))
grave_list = []
cat_names = {}
obj_dict = {}
all_obj = {}
for o in obj_files:
    is_obj = re.search('nextObjectNumber|groundHeat', o) is None
    
    if is_obj:
        o_num = int_extract('[0-9]+(?=.txt)', o)
        o_data = obj.read_obj(o_num)
        if 'Grave' in o_data['name']:
            grave_list = np.append(grave_list, int(o_num))
        obj_dict[o_num] = o_data['mapChance']
        if '@' in o_data['name'] :
            cat_names[o_num] = o_data['name']
        else:
            all_obj[o_num] = o_data['name']
#         craftable_dict[o_num] = o_data['name']

Build a dictionary of {category: children} pairs

In [9]:
l = os.listdir('../../OneLifeData7/categories')
cat_arr = [int_extract('[0-9]+(?=.txt)', o) for o in l]
len(cat_arr)

230

In [ ]:
perhaps_list = list()
for i in all_obj.values():
    if 'Perhaps' in i:
        perhaps_list.append(find_parent(all_obj, i))
perhaps_list

In [1747]:
#update: if parent is not a @category name, add parent name to child list
cat_dict = {}
for i in cat_arr:
    children = cat.cat_children(i)
    if (i in cat_names):
        cat_dict[i] = children  
    else:
        cat_dict[i] = list(np.append(i, children))
          
    
len(cat_dict)

230

### Build a dataframe of transitions:

In [12]:
tech_df = pd.DataFrame()
for i in trans_files:
    a = transition.read_transition(opj(trans_dir, i))
    tech_df = pd.concat([tech_df, pd.DataFrame.from_records(a, index=[0])], sort=True)
tech_df.head()

,actorMinUseFraction,autoDecaySeconds,desiredMoveDist,isTool,lastUseActor,lastUseTarget,move,newActor,newActorName,newTarget,newTargetName,noUseActor,noUseTarget,origActor,origActorName,origTarget,origTargetName,reverseUseActor,reverseUseTarget,targetMinUseFraction
0,0.0,10,1.0,False,False,False,0.0,0,Empty,2578,Cool Glass,0.0,0.0,-1,Empty,2574,Molten Glass,0.0,0.0,0.0
0,0.0,0,1.0,False,False,False,0.0,425,Wolf Skin,695,Wolf Crown,NaN,NaN,0,Empty,702,Wolf Crown with Wolf Skin,0.0,0.0,0.0
0,NaN,0,NaN,True,False,False,NaN,0,Empty,317,Crucible with Iron,NaN,NaN,314,Wrought Iron,235,Clay Bowl - empty,NaN,NaN,NaN
0,0.0,0,1.0,True,False,False,0.0,235,Clay Bowl - empty,3699,Bowl with Raw Rubber Balls,0.0,0.0,2165,Bowl with Raw Rubber Ball,2165,Bowl with Raw Rubber Ball,0.0,1.0,0.0
0,0.0,0,1.0,False,False,False,0.0,1719,Hungry Schnauser Puppy - held,1706,Schnauser with Puppies -2,0.0,0.0,0,Empty,1692,Schnauser with Puppies -3,0.0,0.0,0.0


Only keeps origActor, origTarget, newActor, newTarget

In [1473]:
tech_tree_df = tech_df[['origActor','origTarget','newActor','newTarget']].reset_index(drop = True)
tech_tree_df.head()

,origActor,origTarget,newActor,newTarget
0,-1,2574,0,2578
1,0,702,425,695
2,314,235,0,317
3,2165,2165,235,3699
4,0,1692,1719,1706


### expand on categories

In [1780]:
tech_tree_newdf = tech_tree_df.copy().astype('object')
for i in range(0,len(tech_tree_newdf)):
    for j in ['origActor','origTarget','newActor','newTarget']:
        curr_cell = tech_tree_df.iloc[i][j]
        if curr_cell in cat_dict:
            
            tech_tree_newdf.at[i,j] = list(cat_dict[curr_cell])


In [1781]:
tech_tree_newdf = tech_tree_newdf.query('(newActor != 0) or (newTarget != 0)') #get rid of products = [0,0]

In [1782]:
tech_tree_newdf.loc[tech_tree_newdf.newTarget == tech_tree_newdf.origActor, 'newTarget'] = ''
tech_tree_newdf.loc[tech_tree_newdf.newTarget == tech_tree_newdf.origTarget, 'newTarget'] = ''
tech_tree_newdf.loc[tech_tree_newdf.newActor == tech_tree_newdf.origTarget, 'newActor'] = ''
tech_tree_newdf.loc[tech_tree_newdf.newActor == tech_tree_newdf.origActor, 'newActor'] = ''

In [1783]:
tech_tree_newdf.head(n = 20)

,origActor,origTarget,newActor,newTarget
0,-1,2574,0,2578
1,0,702,425,695
2,314,235,0,317
3,2165,2165,235,3699
4,0,1692,1719,1706
5,0,3085,2228,3086
6,-1,2206,0,2208
7,455,504,0,811
8,1619,2563,0,2564
9,0,1686,1717,1696


In [1785]:
len(tech_tree_newdf)

4302

In [1787]:
[cat_dict[i] for i in perhaps_list]

[[2328, 2304, 2306],
 [3221, 1196, 3220],
 [3233, 1196, 3220],
 [2095, 2092, 2091, 2094, 2097, 2098],
 [2811, 2813, 2812, 2824]]

In [1800]:
tech_df.query('(newTarget in @perhaps_list) or (newActor in @perhaps_list)')

,actorMinUseFraction,autoDecaySeconds,desiredMoveDist,isTool,lastUseActor,lastUseTarget,move,newActor,newActorName,newTarget,newTargetName,noUseActor,noUseTarget,origActor,origActorName,origTarget,origTargetName,reverseUseActor,reverseUseTarget,targetMinUseFraction
0,0.0,60,1.0,False,False,False,0.0,0,Empty,3221,Perhaps a Pumpkin,0.0,0.0,-1,Empty,1195,Blooming Squash Plant,0.0,0.0,0.0
0,0.0,60,1.0,False,False,False,0.0,0,Empty,3233,Perhaps a Squash,0.0,0.0,-1,Empty,3232,Blooming Pumpkin Plant,0.0,0.0,0.0
0,0.0,5,1.0,False,False,False,0.0,0,Empty,2095,Perhaps a Fish,0.0,0.0,-1,Empty,2101,Cast Fishing Pole,0.0,0.0,0.0
0,0.0,10,1.0,False,False,False,0.0,0,Empty,2328,Perhaps Oil,0.0,0.0,-1,Empty,2303,Firing Oil Drilling Rig,0.0,0.0,0.0
0,0.0,10,1.0,False,False,False,0.0,0,Empty,2328,Perhaps Oil,0.0,0.0,-1,Empty,2331,Firing Oil Drilling Rig - extended,0.0,0.0,0.0
0,0.0,5,1.0,False,False,False,0.0,0,Empty,2811,Perhaps some Shrimp,0.0,0.0,-1,Empty,2810,Cast Net,0.0,0.0,0.0


In [1789]:
mydf = tech_tree_newdf.copy()
edge_cases = []
newdf = pd.DataFrame(columns = ['origActor','origTarget','newActor','newTarget'])
for i in range(0,len(mydf)):
    curr_line = mydf.iloc[i]
    target_columns = []
    for j in ['origActor','origTarget','newActor','newTarget']:
        if type(curr_line[j]) is list:
            target_columns = np.append(target_columns,j)
            
    #nothing in the equation has a category id       
    if (len(target_columns) == 0):
        newdf = newdf.append(curr_line.to_frame().T) #append to dataframe
        
    #category id only in product: category id is actually item id
    elif (len(target_columns) == 1) and ((target_columns=='newTarget') or (target_columns=='newActor')):
        if (target_columns=='newTarget'): #if the product is in target
            if curr_line['newTarget'][0] in perhaps_list: #special cases when the product is 'Perhaps xxx'
                curr_line['newTarget'] = curr_line['newTarget'][1:]
                for k in curr_line.values: #first turn every cell into a list obj
                    if type(k) is int:
                        k = [k]
                sub = split_dataframe_rows(curr_line.to_frame().T, target_columns) #split row
                newdf = newdf.append(sub)#append to dataframe
            else: #normal cases
                curr_line['newTarget'] = curr_line['newTarget'][0] #only keep the first element of the list
                newdf = newdf.append(curr_line.to_frame().T) #append to dataframe
        elif (target_columns=='newActor'): #if the product is in actor
            curr_line['newActor'] = curr_line['newActor'][0]
            newdf = newdf.append(curr_line.to_frame().T) #append to dataframe
        
    #more categories in equation and they match in lengths
    else:
        length = len(curr_line[target_columns[0]])
        if all(len(lst) == length for lst in curr_line[target_columns]): #check that they are of the same lengths
            for k in curr_line.values: #first turn every cell into a list obj
                if type(k) is int:
                    k = [k]
            sub = split_dataframe_rows(curr_line.to_frame().T, target_columns) #split row
            newdf = newdf.append(sub)#append to dataframe
        else:
            edge_cases = np.append(edge_cases, i) #edge cases are not included in newdf
        


In [1790]:
newdf = newdf.query('(newActor != "") or (newTarget != "")') #remove situations where nothing is produced

In [1791]:
len(newdf)

8160

In [1792]:
edge = tech_tree_newdf.iloc[edge_cases].copy()
edge = edge.query('newTarget != 1947') #ignore the cards for now
edge

,origActor,origTarget,newActor,newTarget
275,"[210, 382]","[1802, 1803, 1804, 1805, 1872, 2723, 3069, 4311]",,"[1806, 1809, 1808, 1807, 1873, 2724, 3070, 4312]"
407,"[210, 382]","[1790, 1791, 1799, 1798, 1868, 2721, 3071, 4308]",,"[1792, 1793, 1801, 1800, 1869, 2722, 3072, 4309]"
438,"[210, 382]","[1034, 1056, 1055]",,"[1035, 1057, 1058]"
727,"[135, 560]",1015,,"[1016, 1025, 1026]"
1834,"[139, 850, 857]","[1136, 1101]",,213
1973,59,"[2226, 2245]",0,"[2220, 2240, 2270, 2280, 2314, 2305, 2312, 235..."
2443,"[71, 334, 34, 560]","[2982, 2986, 2985]",,2957
2569,"[912, 1000]","[977, 978, 979, 980, 981, 982, 983, 984]",917,
2595,"[210, 382]","[157, 158, 159, 231]",,127
3448,"[210, 382]","[1017, 1042, 1045]",,"[1018, 1043, 1046]"


In [1793]:
cols = ['origActor','origTarget','newActor','newTarget']
edge_df = pd.DataFrame(columns = cols)

for j in range(0, len(edge)):
    lengths = [1,1,1,1]
    curr_line = edge.iloc[j]
    for i in range(0,len(curr_line.values)):
        if type(curr_line[i]) is list:
            lengths[i] = len(curr_line[i])

    num_repeat = np.product(np.unique(lengths))
    mat = -np.ones([num_repeat, 4]).astype('object')
    for i in range(0,4):
        if lengths[i] != max(lengths):
            mat[:,i] = np.repeat(curr_line[i], num_repeat/lengths[i])
        else:
            mat[:,i] = curr_line[i]*int(num_repeat/lengths[i])
    sub = pd.DataFrame(mat, columns = cols)
    edge_df = edge_df.append(sub)

In [1794]:
edge_df.head(n= 10)

,origActor,origTarget,newActor,newTarget
0,210,1802,,1806
1,210,1803,,1809
2,210,1804,,1808
3,210,1805,,1807
4,210,1872,,1873
5,210,2723,,2724
6,210,3069,,3070
7,210,4311,,4312
8,382,1802,,1806
9,382,1803,,1809


In [1796]:
newdf = newdf.append(edge_df)
len(newdf)

8306

make a list of items

In [1797]:
max_item_id = max(list(all_obj.keys()))
print(max_item_id)

4347


In [1798]:
#replace negative values
newdf = newdf.replace(-1, max_item_id+1)
newdf = newdf.replace(-2, max_item_id+2)
#make a list of all objects
items = np.arange(max_item_id+3)
print(max(items))

4349


### create a nested dictionary of ingredients - products pair

In [1799]:
#dictionary problem has been fixed. Ingredients will not override
#Update: also expands on situations where a category is in one side of the transition
d = {}
transition_empt = {}
for item in all_obj:
    transition_empt[item] = []
    d[item] = {}
    recipy = newdf.query('(origActor == @item) or (origTarget == @item)')

    if len(recipy):
        for j in range(0,len(recipy)):
            ingredients = recipy.iloc[j].tolist()[0:2]
            ingredients.remove(item)
            ingredient = ingredients[0]
            products = recipy.iloc[j].tolist()[2:4]
            if "" in products:
                products.remove("")
            if ingredient in d[item].keys(): #prevent override
                d[item][ingredient] = list(map(int, set(np.append(d[item][ingredient], products))))
            else:
                d[item][ingredient] = products         
                

### search for objects that cannot be crafted (with no ingredients)

In [24]:
#find objects with no ingredients
no_ingredients = []
new_child = []
for i in items:
    if i not in cat_dict: #it's not a category
        ingredients = find_ingredients(i)
        if len(ingredients) == 0: #does not have ingredients
            no_ingredients = np.append(no_ingredients, i)
no_ingredients = no_ingredients.astype(int)


len(no_ingredients)

753

### determine natural objects and add them to roots

In [25]:
nat_obj = []
for item in obj_dict.items():
    if item[1] != 0 :
        nat_obj.append(item[0])
roots = list(set(np.append(nat_obj, no_ingredients)))
len(roots)

800

### loop over all pairs

In [1801]:
#initialize depth
orig_depth = np.inf
depth = {}
for item in items:
    if item in nat_obj:
        depth[item] = 0
    else:
        depth[item] = orig_depth
depth[0] = 0 #empty hand
depth[len(depth)-1] = 0 #empty ground
depth[len(depth)-2] = 0 #empty (for animals)

In [1802]:
root_items = nat_obj
order_list = []
transition_from = transition_empt

while len(root_items):
    #take an item from root and remove it
#     root_items = list(set(root_items)) #remove repeated items
    item = root_items[0]
    root_items = np.delete(root_items,0)
#     print(root_items)

    #put all products associated with this item into root
    for key in d[item]: #looping through other ingredients
        #calculate the depths of the products
        for product in d[item][key]:

            #if this product is made of known-depth ingredients, add this product to root_items list
            if depth[product] > (depth[item] + depth[key] + 1):
                print(item, product)
                depth[product] = depth[item] + depth[key] + 1  

                #if the ingredients have NOT been used in one of the steps of producing the object
#                 if (item not in transition_from[product]):
#                     transition_from[product] = np.append(transition_from[product],\
#                                                          np.append(transition_from[item], item))
#                 if (key not in transition_from[product]):
#                     transition_from[product] = np.append(transition_from[product],\
#                                                          np.append(transition_from[key], key))
                root_items = np.append(root_items, product)
                order_list = np.append(order_list,product)
                

706 717
706 718
100 96
2174 2175
1323 1328
1323 1333
1323 1364
674 2725
1874 1878
4272 4273
4272 4274
729 3973
703 705
63 64
63 48
1435 1436
764 1385
764 1377
2504 3896
49 61
1184 1185
1184 1183
161 166
1157 1160
1157 1158
4239 4240
4239 4242
65 66
65 45
3030 1096
761 762
1020 2407
1020 2412
2515 3897
630 647
3001 3971
418 427
418 1363
4221 4222
4221 4223
1013 1015
1013 1014
805 808
50 57
50 53
50 51
153 132
153 406
531 532
1890 3975
1891 3974
1107 1123
1107 1108
2142 2145
2142 2143
2156 2157
2156 2155
4269 4270
242 223
2800 2801
2800 2803
36 395
36 404
32 34
2804 2805
2804 2806
2804 2807
133 150
33 661
33 3963
33 4236
2540 3972
291 1836
1261 1262
1261 141
4251 4252
125 126
125 409
30 279
30 31
2557 2558
717 3424
718 3423
96 4068
1328 1334
64 67
64 4062
48 62
1185 1188
166 165
1158 1159
4240 4241
66 68
66 2419
1096 3175
762 763
2407 2413
2407 2465
2407 2405
2412 2739
2412 2408
647 648
4222 4235
808 3980
57 58
53 56
51 54
51 52
132 4063
132 131
1123 1128
2143 3977
2143 2144
2803 2802
40

444 446
444 453
451 455
2209 2212
2210 2211
2301 2302
2206 2208
2454 2460
333 334
855 857
446 449
446 447
453 454
455 1888
455 1887
455 1886
455 906
455 862
455 862
2212 2251
2211 2335
2302 4101
2208 2213
2460 2461
2460 3146
334 2768
334 340
334 3311
334 3146
334 342
334 340
334 3311
449 456
447 450
447 448
454 462
906 905
2213 2227
3146 344
3146 341
2768 2773
2768 2770
342 343
456 463
450 457
448 4191
448 452
448 499
462 3271
462 3271
462 556
905 904
344 1316
344 346
341 339
2773 2798
2773 2774
2770 2771
2770 2771
343 338
457 464
4191 1525
4191 4190
452 1526
452 459
499 500
499 682
339 345
2774 2775
2774 2775
2771 2772
338 1268
464 465
1525 1593
1525 4192
459 907
459 2795
500 501
682 683
345 467
345 468
345 1317
2775 2776
1268 1270
1268 1271
465 466
1593 1524
1593 1528
907 910
2795 4315
2795 4314
501 502
683 685
467 1533
467 1531
467 469
467 2886
467 2239
467 1221
467 863
2776 2778
1270 3421
1271 1272
1271 1274
466 458
1524 1527
1528 1529
910 908
502 3144
502 3143
502 496
502 555
502 

1633 1638
1633 1658
1779 1632
1609 1619
1465 1467
2791 2792
2791 2793
3856 3855
3856 3859
3580 3578
3580 3583
3574 3572
3574 3576
3864 3869
3587 3586
3587 3590
2162 2163
1576 1572
1579 1573
1555 1568
1554 1569
2814 2815
2817 2818
2817 4071
4079 4071
1818 1817
1818 1819
1818 1819
1875 1877
4299 4300
1239 1240
2478 2487
2231 2230
818 820
795 793
795 796
3382 3381
3992 3993
1676 1667
1758 1760
1658 3985
1658 1677
1658 1749
1632 1637
1632 1630
1619 2564
1619 2114
1619 1835
1619 1616
1619 2543
1619 3114
1619 3185
1467 1473
2792 2794
3855 3857
3859 3854
3578 3581
3583 3571
3572 3573
3576 3570
3869 3868
3869 3872
3586 3588
3590 3585
2163 2164
1572 1580
1572 1585
1572 3786
1572 3790
1572 3806
1572 3808
1572 1582
1573 1581
1573 1584
1573 3795
1573 3799
1573 3807
1573 3809
1568 1557
1568 1586
1568 3787
1568 3796
1569 1582
1569 1556
1569 1587
1569 3783
1569 3792
2818 2819
1817 1834
1817 1834
1819 1820
1877 1879
1877 3976
4300 4301
1240 1241
2487 2486
2230 2229
820 822
3381 3380
3993 3994
1667 168

In [1803]:
depth.values()

dict_values([0, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, 0, 1, 0, 0, 1, 5, 0, inf, inf, 2, 3, inf, inf, inf, inf, 1, inf, inf, 1, 0, 0, 1, 2, 1, 2, 3, 2, 1, 3, 7, inf, 1, 2, 0, 1, 0, 1, 3, 3, 5, 13, 15, 17, 11, 17, 21, inf, 24, 26, 28, 27, inf, 45, 63, inf, 46, 47, 35, 36, 37, inf, inf, 11, inf, inf, inf, 1, inf, inf, 0, 0, inf, inf, inf, inf, inf, 13, 7, 9, 10, 11, 25, 23, 24, 19, 29, 30, 31, inf, 30, inf, 0, 3, 2, 3, 0, 1, 5, 11, inf, inf, 3, 1, 0, 5, 3, 0, 3, 2, 3, 7, 1, 3, 2, 3, inf, 7, 13, 21, 15, 2, 11, 33, 0, 23, 21, 22, 17, 15, 16, 15, 0, 17, 20, 3, 2, 1, 5, 26, 25, inf, 29, 28, 4, 27, 30, 28, 31, 29, 32, 29, 34, 33, 34, 38, 38, 85, 84, inf, 88, 87, 92, 96, 6, 7, 30, 31, 86, 276, 166, 131, 201, 236, 136, 69, 104, 139, 39, 179, 135, 136, 0, 277, 13, 18, 118, 19, 119, 119, 120, 4, inf, inf, 1, 2, 3, 5, 6, 15, 115, 116, 6, inf, 2, 3, 98, 99, 12, 18, 7, 11, 98, 0, 117, 3, 104, in

In [1804]:
not_updated = find_value(depth, np.inf, 0)
len(not_updated)

1009

In [1805]:
for i in not_updated:
    try:
        print(obj.read_obj(str(i))['name'])
    except:
        SyntaxError

Skin Tone A &B &C &D &E &F
Female001 D
Milkweed Stump -Fertile
Male002 D
Female003 D
Female004 C
Male005 D
Female006 C
Male007 C
Male008 C
Touched Edge
Edge
@ Kindling Source
@ Pond Water Source
@ Full Portable Water Source
@ Carrot
@ Wet Canada Goose Pond
Jason Test
Long Play Phonorecord Album
Phonorecord Sleeve
Long Play Phonorecord
Phonograph
Playing Phonograph
@ Yew Bow
Flooded Pond
@ Food Burner
@ Edible Pie
@ Skinning Tool
@ Lamb Food Bowl
@ Small Trash
@ Planted Stakes
Parts
@ Unshot Grizzly Bear
@ Empty Portable Water Source
@ New Scrap Steel
@ Shallow Digger
@ Rough Cutter
@ Simmering Liquid
@ Partial Adobe Build
@ Stone Rubble Sources
Floppy Basket
@ Row Tiller
@ Non-tilling Skewer
Broken Basket
@ Decaying Hat
@ Decaying Shirt
@ Decaying Bottom
@ Decaying Shoe
@ Tillable Row
@ Decaying Basket
@ Broken Steel Source
@ Weak Chisel Breaker
Fence Rail
@ Fine Cutter
@ Weak Froe Breaker
@ Unlocked Sign
@ Locked Sign
@ Free Lock
Female009 D
Male010 D
Female011 C
Male012 C
Full Deep W

In [1807]:
hardest_item = max([i for i in list(depth.values()) if i < np.inf])
myitem = find_value(depth, hardest_item, 0)[0]
tech_df.query('(newTarget == @myitem) or (newActor == @myitem)')

,actorMinUseFraction,autoDecaySeconds,desiredMoveDist,isTool,lastUseActor,lastUseTarget,move,newActor,newActorName,newTarget,newTargetName,noUseActor,noUseTarget,origActor,origActorName,origTarget,origTargetName,reverseUseActor,reverseUseTarget,targetMinUseFraction
0,0.0,0,1.0,True,False,False,0.0,0,Empty,4208,Northwestward Gradient Dry Tarry Spot,0.0,0.0,4207,Southwestward Gradient Dry Tarry Spot,4207,Southwestward Gradient Dry Tarry Spot,0.0,0.0,0.0


In [1808]:
depth[myitem]

33599103

### Scratch

In [1810]:
for i in depth:
    l = list()
    if depth[i] == np.inf:
        ing = find_ingredients(i)
        for j in ing:
            if depth[j] != np.inf:
                l.append(j)
        if len(l)>=2:
            print(i)

2960
2961
2977
2978
2985
2986
3095
3096
3120
3122
3151
3152
3154


In [84]:
# #approach 2 where ingredients and products are all expanded into items rather than category
# root_items = nat_obj
# iterated_items = []
# order_list = []
# transition_from = transition_empt

# while len(root_items):
#     #take an item from root and remove it
#     root_items = list(set(root_items)) #remove repeated items
#     item = root_items[0]
#     root_items = np.delete(root_items,0)
#     while item in cat_dict:
#         sub_item = cat_dict[item]
#         root_items = np.append(root_items, sub_item)
#         item = root_items[0]
#         root_items = np.delete(root_items,0)
#     iterated_items = np.append(iterated_items, item)
    

#     #put all products associated with this item into root
#     for key in d[item]: #looping through other ingredients
# #         if key == 'category' :
# #             keys = d[item][key]
# #             kidx = np.argmin([depth[k] for k in keys])
# #             key = keys[kidx]

#         #calculate the depths of the products
#         for product in list(set(d[item][key])):
#             #if this product is made of known-depth ingredients, add this product to root_items list
#             if depth[product] > (depth[item] + depth[key] + 1):
#                 depth[product] = depth[item] + depth[key] + 1
# #                 if product in cat_dict: #if this is a category, push all its' children to root_items
# #                     for c in cat_dict[product]:
# #                         depth[c] = min(depth[c], depth[product])
# #                         if c not in iterated_items: #avoid repeated iterating over the same item
# #                             root_items = np.append(root_items, c)
# #                 else:
# #                     root_items = np.append(root_items, product)
# #                     print(product, cat_dict[product])    

#                 #if the ingredients have NOT been used in one of the steps of producing the object
#                 if (item not in transition_from[product]):
#                     transition_from[product] = np.append(transition_from[product],\
#                                                          np.append(transition_from[item], item))
#                 if (key not in transition_from[product]):
#                     transition_from[product] = np.append(transition_from[product],\
#                                                          np.append(transition_from[key], key))
#                 root_items = np.append(root_items, product)
#                 order_list = np.append(order_list,product)
                

In [23]:
# #dictionary problem has been fixed. Ingredients will not override
# #Update: also expands on situations where a category is in one side of the transition
# d = {}
# transition_empt = {}
# for item in items:
#     transition_empt[item] = []
#     if item in cat_dict:
#         d[item] = {'category':cat_dict[item]} #category item indicated by key
#     else:
#         d[item] = {}
#         recipy = tech_tree_newdf.query('(origActor == @item) or (origTarget == @item)')
#         if len(recipy):
#             for j in range(0,len(recipy)):
#                 ingredients = recipy.iloc[j].tolist()[0:2]
#                 ingredient = np.delete(ingredients, np.where(ingredients == item)[0][0])[0] 
#                 products =[]
#                 for p in recipy.iloc[j].tolist()[2:4]:
#                     if (p!=item) and (p!=ingredient): #remove unchanged item from product
#                         if p in cat_dict: #expand on categories in product: find the product associated with the item
#                             products = np.append(products, cat_dict[p])
# #                             print(products)
#                         else:
#                             products = np.append(products, p)
#                         products = np.append(products, p)
#                 products = list(map(int, products))
                
#                 if ingredient in cat_dict: #expand on categories in key ingredient
#                     all_ingredients = cat_dict[ingredient]
#                 else:
#                     all_ingredients = [ingredient]
#                 for t in all_ingredients: 
#                     #this is to prevent that transitions with the same ingredients but different products override
#                     if t in d[item].keys():
#                         d[item][t] = list(map(int, set(np.append(d[item][t], products))))
#                     else:
#                         d[item][t] = products          
                

In [308]:
empt = []
for i in transition_from.keys():
    if len(transition_from[i]) == 0:
        if (i not in roots) and (i not in cat_dict) and (i not in grave_list):
            empt = np.append(empt, i)
empt = empt.astype(int)
len(empt)

271

In [310]:
most_ingredients = np.argmax([len(transition_from[i]) for i in transition_from.keys()])
len(transition_from[most_ingredients])

562583